In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium 
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from glob import glob
import os
pbar = ProgressBar()
pbar.register()

from tqdm import tqdm_notebook as tqdm

### 행정동

In [ ]:
hdong_df = pd.read_csv('./data/행정구역분류와 행정기관 및 법정동코드 연계표(2018.4.1.기준).csv',encoding= 'cp949')
hdong_df = hdong_df.iloc[:,:10]
gwangjin = hdong_df.loc[hdong_df.시군구 == '광진구']

# #임시저장
# gwangjin.to_csv('./data/광진구_행정동코드.csv',index= False, encoding = 'cp949')

#8자리 행정동 코드로 변경
gwangjin_dong =gwangjin['행정기관코드(행정안전부 행정동코드)'].apply(lambda x : int(str(x)[:-2])).values

### loop 구현

In [ ]:
path = 'F:/data/'
first_folders = os.listdir(path)

In [ ]:
%%time
#예상소요시간 개당 10분 -> 12개 약 120분 
#utf-8에러 뜨면, isin에 연산하는 값의 데이터타입이 동일한지 확인할것. 빈값이여서 오류날수 있음
for folder in tqdm(first_folders[:-1]):
    globs = glob(path + folder+'/*.csv')
    #resi만
    for g in globs[:1]:
        filename = g.split('\\')[1].split('.')[0]
        pre = filename.split('_')[0]
        if pre == 'resi':
            resi_df = dd.read_csv(g,sep =',',assume_missing=True,
                                 names = ['ID','X_AXIS',"Y_AXIS","SEX","AGE","WEEKDAY","WEEKEND","FROM_ID",'DONG_ID','YYYYMM'],
                                 header= None)
            resi_kt_gwangjin = resi_df.loc[resi_df.DONG_ID.isin(gwangjin_dong)].compute(scheuler = 'processes', num_worker = 16)
            os.makedirs('E:/python/data/KT 데이터1/', exist_ok=True)
            resi_kt_gwangjin.to_csv(f'E:/python/data/KT 데이터1/{filename}.csv',index = False)

### 데이터가 이상하다?? 201705 전용
tail이 안됨.. 뭐야 -> 파일깨졌었음..?? 201705만 이상함  
ㅁ과 같은 이상한 문자가 들어가 있음 -> dtype 지정 및 encoding 방식 바꿔서 해결

In [ ]:
resi_df = dd.read_csv('F:/data/201705/resi_201705.csv',sep =',',assume_missing=True,
                     names = ['ID','X_AXIS',"Y_AXIS","SEX","AGE","WEEKDAY","WEEKEND","FROM_ID",'DONG_ID','YYYYMM'],
                     header= None ,encoding = 'latin1',
                     dtype = {'ID': str})
resi_kt_gwangjin = resi_df.loc[resi_df.DONG_ID.isin(gwangjin_dong)].compute(scheuler = 'processes', num_worker = 16)
os.makedirs('E:/python/data/KT 데이터1/', exist_ok=True)
resi_kt_gwangjin.to_csv(f'E:/python/data/KT 데이터1/{filename}.csv',index = False)

In [ ]:
%%time
#wlk만 개당 약 2분 약 30분 소요 
for folder in tqdm(first_folders):
    globs = glob(path + folder+'/*.csv')
    for g in globs[1:]:
        filename = g.split('\\')[1].split('.')[0]
        wlk_df = dd.read_csv(g,sep =',',assume_missing=True,
                 names = all_names,
                 header= None)
        walk_kt_gwangjin = wlk_df.loc[wlk_df.DONG_ID.isin(gwangjin_dong)].compute(scheuler = 'processes', num_worker = 16)
        os.makedirs('E:/python/data/KT 데이터/', exist_ok=True)
        walk_kt_gwangjin.to_csv(f'E:/python/data/KT 데이터/{filename}.csv',index = False)
